In [5]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [6]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils



In [7]:
asset_path="../asset/tapas_wikisql_sqa_masklm_mini/"

In [8]:
os.makedirs(asset_path+"results/sqa/tf_examples", exist_ok=True)
os.makedirs(asset_path+"results/sqa/model", exist_ok=True)
with open(asset_path+'checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'{asset_path}model.ckpt{suffix}', f'{asset_path}results/sqa/model/model.ckpt-0{suffix}')

In [9]:
data=pd.read_csv("test.csv")
data.head()
data=data.loc[:20,:]
data=data.astype(str)
list_data=[[]]
list_data[0]=list(data.columns)
list_data.extend(data.values.tolist())


In [16]:
max_seq_length = 512
vocab_file = asset_path+"vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table =table_data# [list(map(lambda s: s.strip(), row.split("|"))) 
         #  for row in table_data.split("\n") if row.strip()]
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example(asset_path+"results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example(asset_path+"results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  !python -m tapas.run_task_main.py \
    --task=asset_path+"SQA" \
    --output_dir=asset_path+"results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity=asset_path+"ERROR" \
    --compression_type= "tsv"\
    --init_checkpoint=asset_path+"model.ckpt" \
    --bert_config_file=asset_path+"bert_config.json" \
    --mode=asset_path+"predict" 2> error


  results_path = asset_path+"results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [17]:
# Example nu-1000-0
result = predict(list_data, ["what were the passengers names?"])

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,nan,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0,nan,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,nan,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,nan,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,nan,S
897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.225,nan,S
898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,nan,Q
899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0,nan,S
900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,nan,C
901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.15,nan,S


FileNotFoundError: [Errno 2] No such file or directory: '../asset/tapas_wikisql_sqa_masklm_mini/results/sqa/model/test_sequence.tsv'